In [98]:
import pandas as pd
import numpy as np
import os,re
from tqdm.notebook import tqdm_notebook as tqdm

sighan05 = "../第二届国际中文分词评测/icwb2-data/" # 语料库
msr_train = os.path.join(sighan05, 'training', 'msr_training.utf8') # 训练集
msr_trainLabel = os.path.join(sighan05, 'training', 'msr_trainLabel.txt') # 自建标注集
msr_test = os.path.join(sighan05, 'testing', 'msr_test.utf8') # 测试集
msr_output = os.path.join(sighan05, 'testing', 'msr_output_4.txt') # 保存输出结果的空文件
msr_gold = os.path.join(sighan05, 'gold', 'msr_test_gold.utf8') # 测试集的正确结果

OUTFRMAE = "P:\t%s\nR:\t%s\nF1:\t%s" # 准确性结果的输出格式

In [2]:
import re,string
from zhon.hanzi import punctuation

In [3]:
def word_labeling(word):
    pos_list = []   
    n = len(word)
    if n == 1:
        pos_list.append('S')
    elif n==2:
        pos_list.extend(list('BE'))
    else:
        pos_list.extend(list('B'+(n-2)*'M'+'E'))
    return pos_list    

In [4]:
from prob_start import P as start_p # 状态初始概率
from prob_trans import P as trans_p # 状态转移概率
from prob_emit import P as emit_p   # 状态发射概率 # tabular

In [5]:
count_S = 0
count_B = 0

In [78]:
with open(msr_train, encoding='utf-8') as train, \
     open(msr_trainLabel, 'w', encoding='utf-8') as trainlabel:
    sentence_list = [item[:-1].split(' ') for item in train]
    for i in tqdm(range(len(sentence_list))):
        sentence_list[i] = sum(
            [list(zip(word, word_labeling(word))) for word in sentence_list[i] if len(word)>0],[])
        for w,l in sentence_list[i]:             
            trainlabel.write("\t".join([w,l])+'\n')
        trainlabel.write("\n")

In [65]:
with open(msr_train, encoding='utf-8') as train:
    for line in tqdm(train):
        word = line.split("  ")[0]
        if len(word)==1:
            count_S += 1   
        else:
            count_B += 1   

In [66]:
count_B

75765

In [67]:
count_S

32234

In [68]:
from math import log
start_p['B'] = log(count_B/(count_S+count_B))
start_p['S'] = log(count_S/(count_S+count_B))

In [69]:
start_p

{'B': -0.3544855232339677,
 'E': -3.14e+100,
 'M': -3.14e+100,
 'S': -1.209100171721567}

In [79]:
df = pd.read_table(msr_trainLabel,encoding="utf-8",error_bad_lines=False,header=None)

b'Skipping line 2685079: expected 2 fields, saw 3\n'


In [83]:
df[1].value_counts()

E    1255055
B    1255055
S    1113336
M     427023
Name: 1, dtype: int64

In [84]:
pos_list = list(df[1].values)
pos_new_list = [] #绑定挨着的两个字母
for i in range(len(pos_list)-1):
    pos_new_list.append((pos_list[i]+pos_list[i+1]))
from collections import Counter
count_label = dict(Counter(pos_list))
count_labelcom = dict(Counter(pos_new_list))
for i in count_labelcom:
    pre_i = i[0]
    lat_i = i[1]
    trans_p[pre_i][lat_i] = np.log(count_labelcom[i]/count_label[pre_i]) #计算概率的对数值

In [85]:
trans_p

{'B': {'E': -0.18804907187170708, 'M': -1.763603863953054},
 'E': {'B': -0.7461934539962262, 'S': -0.64277363126905},
 'M': {'E': -0.6855070645928693, 'M': -0.7008461175870558},
 'S': {'B': -0.5229551484715934, 'S': -0.8983701271010573}}

In [86]:
for i in 'BMES':
    emit_p[i] = dict(np.log(df[df.loc[:,1]==i].value_counts(normalize=True)))

In [87]:
emit_p

{'B': {('一', 'B'): -4.047717711630636,
  ('１', 'B'): -4.443081609107167,
  ('中', 'B'): -4.449607361311863,
  ('不', 'B'): -4.591680504132731,
  ('国', 'B'): -4.61313373433653,
  ('发', 'B'): -4.619579651426111,
  ('人', 'B'): -4.868080392268559,
  ('经', 'B'): -4.868287689118389,
  ('大', 'B'): -4.895182891484329,
  ('这', 'B'): -4.932390936492986,
  ('工', 'B'): -4.986433597695474,
  ('全', 'B'): -5.099183822251702,
  ('进', 'B'): -5.100490499557705,
  ('２', 'B'): -5.144734267404912,
  ('有', 'B'): -5.18063127689347,
  ('我', 'B'): -5.226578058494936,
  ('成', 'B'): -5.236116133785343,
  ('政', 'B'): -5.263594143408412,
  ('实', 'B'): -5.309579256650235,
  ('自', 'B'): -5.345845434634629,
  ('开', 'B'): -5.350031993541964,
  ('重', 'B'): -5.355753988285611,
  ('建', 'B'): -5.366614437812514,
  ('主', 'B'): -5.371231803861276,
  ('生', 'B'): -5.37793919851509,
  ('出', 'B'): -5.450018301696799,
  ('提', 'B'): -5.484162899379728,
  ('学', 'B'): -5.4885864088526075,
  ('地', 'B'): -5.496132154242801,
  ('以', 'B'

In [100]:
def viterbi(obs, states, start_p, trans_p, emit_p):
    V = [{}]  # tabular
    path = {}
    for y in states:  # init
        V[0][y] = start_p[y] + emit_p[y].get(obs[0], MIN_FLOAT)
        path[y] = [y]
    #print(path)
    for t in range(1, len(obs)):
        V.append({})
        newpath = {}
        for y in states:
            em_p = emit_p[y].get(obs[t], MIN_FLOAT)
            (prob, state) = max(
                [(V[t - 1][y0] + trans_p[y0].get(y, MIN_FLOAT) + em_p, y0) for y0 in PrevStatus[y]])
            V[t][y] = prob
            newpath[y] = path[state] + [y]
        path = newpath
        #print(path)
    (prob, state) = max((V[len(obs) - 1][y], y) for y in 'ES')
    return (prob, path[state])

In [89]:
MIN_FLOAT = -3.14e100

PrevStatus = {
    'B': 'ES',
    'M': 'MB',
    'S': 'SE',
    'E': 'BM'
}

In [103]:
sentence = "商品和服务"
viterbi(sentence, "BMES", start_p, trans_p, emit_p)

(-1.57e+101, ['S', 'S', 'S', 'S', 'S'])

In [92]:
def __cut(sentence):
    global emit_P
    prob, pos_list = viterbi(sentence, 'BMES', start_p, trans_p, emit_p)
    begin, nexti = 0, 0
    # print pos_list, sentence
    for i, char in enumerate(sentence):
        pos = pos_list[i]
        if pos == 'B':
            begin = i
        elif pos == 'E':
            yield sentence[begin:i + 1]
            nexti = i + 1
        elif pos == 'S':
            yield char
            nexti = i + 1
    if nexti < len(sentence):
        yield sentence[nexti:]
print(list(__cut(sentence)))

['商', '品', '和', '服', '务']


In [101]:
import time
with open(msr_test,encoding="utf-8") as test, open(msr_output, 'w', encoding="utf-8") as output:
    start = time.time()
    for line in test:
        sentence = line.strip()
        output.write(" ".join(list(__cut(sentence))))
        output.write("\n")
    runtime = time.time()-start
print('runtime:',runtime)

runtime: 5.883130788803101


In [96]:
import re
re_eng = re.compile('[a-zA-Z0-9]', re.U)

def to_region(segmentation: str) -> list: 
    """
    将分词结果转换为区间

    """
    region = []
    start = 0
    for word in re.compile("\\s+").split(segmentation.strip()):
        end = start + len(word)
        region.append((start, end))
        start = end
    return region

def prf(gold: str, pred: str) -> tuple: # 准确率函数
    """
    计算P、R、F1
   
    """
    A_size, B_size, A_cap_B_size, = 0, 0, 0
    with open(gold, encoding="utf-8") as gd, open(pred, encoding="utf-8") as pd:
        for g, p in zip(gd, pd):  # 取出答案：g和预测：p
            A, B = set(to_region(g)), set(to_region(p))  # 得到区间
            A_size += len(A)
            B_size += len(B)
            A_cap_B_size += len(A & B)
            text = re.sub("\\s+", "", g)  # 得到原始文本
    p, r = A_cap_B_size / B_size * 100, A_cap_B_size / A_size * 100
    return p, r, 2 * p * r / (p + r)

In [102]:
print(OUTFRMAE % prf(msr_gold, msr_output))

P:	26.082286892137454
R:	44.99171914328222
F1:	33.02155012567473
